In [1]:
from classifiers import *

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
import numpy as np

## Model Retraining

In [ ]:
train_datagen = ImageDataGenerator(samplewise_center=True)
valid_datagen = ImageDataGenerator(samplewise_center=True)

train_dir = 'deepfake_database/train_test'
valid_dir = 'deepfake_database/validation'

train_generator = train_datagen.flow_from_directory(
    train_dir,  # This is the target directory
    target_size=(256, 256),  # All images will be resized to 256x256
    batch_size=64,
    class_mode='binary'
    )  # Since you use binary_crossentropy loss, you need binary labels

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,  # This is the target directory
    target_size=(256, 256),  # All images will be resized to 256x256
    batch_size=32,
    class_mode='binary',
    # shuffle=False
    )  

In [ ]:
# To check the classes indices
print(train_generator.class_indices)
print(valid_generator.class_indices)

In [2]:
model = MesoInception4()
model.load('weights/M4I_DF_retrained_galma_best.h5')

# model = Meso4()
# model.load('weights/Meso4_DF.h5')

In [ ]:
model.model.summary()

In [11]:
early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=10, verbose=1, mode='max', restore_best_weights=True)

In [ ]:
history = model.model.fit(
      train_generator,
      steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Automatically set steps based on dataset size
      epochs=15, 
      validation_data=valid_generator,
      validation_steps=valid_generator.samples // valid_generator.batch_size,  # Automatically set steps based on dataset size
      verbose=1,
      callbacks=[early_stopping]
      )

In [ ]:
# Evaluate on training data
train_loss, train_accuracy = model.model.evaluate(train_generator)
print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)

# Evaluate on validation data
valid_loss, valid_accuracy = model.model.evaluate(valid_generator)
print("Validation Loss:", valid_loss)
print("Validation Accuracy:", valid_accuracy)

## Visualization

In [ ]:
plt.bar(['Training', 'Validation'], [train_accuracy, valid_accuracy], color=['orange', 'pink'])
plt.title('Accuracy Comparison')
plt.ylabel('Accuracy')

In [ ]:
# Summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

### Faces Mean Output

In [ ]:
layer_name = 'batch_normalization_7'  # Name of the last conv layer 
intermediate_layer_model = Model(inputs=model.model.input,
                                 outputs=model.model.get_layer(layer_name).output)

# datagen = ImageDataGenerator(rescale=1./255)
datagen = ImageDataGenerator(samplewise_center=True)

# Create a generator for the images
generator = datagen.flow_from_directory(
    # directory='deepfake_database/train_test',  # replace with your directory path
    # directory = 'deepfake_database_v2/train_test',
    directory='test_images_v2',
    # directory = 'deepfake_database_v2/validation',
    target_size=(256, 256),  # target size as per your model's input
    batch_size=64,  # can be set to any size that fits your memory
    class_mode=None,  # since we don't need the labels for activation extraction
    shuffle=False  # you don't need to shuffle for this purpose
)

activations = intermediate_layer_model.predict(generator, steps=len(generator))

# Initialize an array to accumulate the activations
accumulated_activations = []

# Process the images and collect the activations
for i in range(len(generator)):  
    images = generator.next()
    activations = intermediate_layer_model.predict(images)
    accumulated_activations.append(activations)

# Now concatenate the list of activations into one array
all_activations = np.concatenate(accumulated_activations, axis=0)

# Calculate the mean across all batches for each filter
mean_activations = np.mean(all_activations, axis=0)

# mean_activations is now an array of the same shape as the activation outputs of the layer,
# but each activation is the mean activation for that filter across all images
# Number of filters in the convolutional layer
num_filters = mean_activations.shape[-1]

# Set the number of filters you want to display
num_filters_to_display = min(num_filters, 64)  # for example, to display up to 64 filters

# Set the grid dimensions for the subplots
grid_dim = int(np.ceil(np.sqrt(num_filters_to_display)))

# Create a figure to hold the subplots
fig, axes = plt.subplots(grid_dim, grid_dim, figsize=(20, 20))

# Iterate over all the filters
for i in range(num_filters_to_display):
    # Get the mean activation for the i-th filter
    filter_activation = mean_activations[:, :, i]
    
    # Find the row and column we're on
    row = i // grid_dim
    column = i % grid_dim
    
    ax = axes[row, column]
    
    # Display the mean activation for this filter
    ax.imshow(filter_activation, cmap='viridis')
    
    ax.axis('off')

# If the number of filters is not a perfect square, turn off the axes for empty subplots
for i in range(num_filters_to_display, grid_dim * grid_dim):
    axes.flatten()[i].axis('off')
plt.show()

In [40]:
# import os
# print(os.getcwd())

c:\Users\chelo\Desktop\Intro to Machine Learning\Final Project - ML\MesoNet


## Model Saving

In [18]:
model.model.save('weights/M4I_DF_retrained_galma_best.h5')